In [1]:
import pandas as pd
import numpy as np

#from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model che

In [ ]:


df = pd.read_csv("/content/drive/MyDrive/train.csv")
df = df.iloc[:, 0:3]

df.columns = ["classIndex", "title", "description"]
df_cleaned = df.dropna(how='any')
df['description'] = df['description'].fillna("").astype(str)
description = df['description'].tolist()
train_encoded = tokenizer(description, padding=True,  truncation=True,max_length=372 , return_tensors="pt")

In [ ]:
input_ids = train_encoded["input_ids"]
input_ids_array = input_ids.cpu().numpy().astype(np.int16)
np.savez_compressed("train_compressed.npz", trainX=input_ids_array)

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/test.csv")
df2 = df2.iloc[:, 0:3]

df2.columns = ["classIndex", "title", "description"]
df2_cleaned = df2.dropna(how='any')
df2['description'] = df2['description'].fillna("").astype(str)
description2 = df2['description'].tolist()
test_encoded = tokenizer(description2, padding='max_length', truncation=True, max_length=372, return_tensors="pt")

In [ ]:
input_ids2 = test_encoded["input_ids"]
input_ids2_array = input_ids2.cpu().numpy().astype(np.int16)
np.save('testX.npy', input_ids2)

In [ ]:
train_y = df["classIndex"].to_list()
test_y = df2["classIndex"].to_list()
print(train_y)
print(test_y)

In [ ]:
print(len(description), len(description2))
print(len(test_y), len(train_y))

In [ ]:
np.savez_compressed("trainY.npz", trainY=train_y)
np.savez_compressed("testY.npz", testY=test_y)

In [13]:
def one_hot_encoding(value):
    take_zeros = np.zeros(4, dtype=int)
    take_zeros[value - 1] = 1
    return take_zeros

In [14]:
trainY = np.load("../../data/ag_news_dataset/trainY.npy")
testY = np.load("../../data/ag_news_dataset/testY.npy")

In [15]:
print(trainY)

[3 3 3 ... 2 2 2]


In [16]:
print(testY)

[3 4 4 ... 2 3 3]


In [18]:
trainY = np.asarray([one_hot_encoding(val) for val in trainY])
testY = np.asarray[one_hot_encoding(val) for val in testY])